In [24]:
import sqlite3

conn = sqlite3.connect('ds_salaries.db')

file_path = '/Users/hannasharifi/Downloads/ds_salaries.csv'
data = pd.read_csv(file_path)

In [25]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS salaries (
    work_year INTEGER,
    experience_level TEXT,
    employment_type TEXT,
    job_title TEXT,
    salary INTEGER,
    salary_currency TEXT,
    salary_in_usd INTEGER,
    employee_residence TEXT,
    remote_ratio INTEGER,
    company_location TEXT,
    company_size TEXT
)
'''

conn.execute(create_table_query)

df.to_sql('salaries', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

In [7]:
categorical_columns = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 
                       'employee_residence', 'company_location', 'company_size']

unique_values = {column: data[column].unique() for column in categorical_columns}
unique_values

{'experience_level': array(['SE', 'MI', 'EN', 'EX'], dtype=object),
 'employment_type': array(['FT', 'CT', 'FL', 'PT'], dtype=object),
 'job_title': array(['Principal Data Scientist', 'ML Engineer', 'Data Scientist',
        'Applied Scientist', 'Data Analyst', 'Data Modeler',
        'Research Engineer', 'Analytics Engineer',
        'Business Intelligence Engineer', 'Machine Learning Engineer',
        'Data Strategist', 'Data Engineer', 'Computer Vision Engineer',
        'Data Quality Analyst', 'Compliance Data Analyst',
        'Data Architect', 'Applied Machine Learning Engineer',
        'AI Developer', 'Research Scientist', 'Data Analytics Manager',
        'Business Data Analyst', 'Applied Data Scientist',
        'Staff Data Analyst', 'ETL Engineer', 'Data DevOps Engineer',
        'Head of Data', 'Data Science Manager', 'Data Manager',
        'Machine Learning Researcher', 'Big Data Engineer',
        'Data Specialist', 'Lead Data Analyst', 'BI Data Engineer',
        'Dire

In [8]:
Q1 = data['salary_in_usd'].quantile(0.25)
Q3 = data['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = data[(data['salary_in_usd'] < lower_bound) | (data['salary_in_usd'] > upper_bound)]

outliers_summary = outliers['salary_in_usd'].describe()
outliers_count = outliers.shape[0]

lower_bound, upper_bound, outliers_count, outliers_summary

(-25000.0,
 295000.0,
 63,
 count        63.000000
 mean     331394.301587
 std       41264.614303
 min      297300.000000
 25%      300000.000000
 50%      310000.000000
 75%      350000.000000
 max      450000.000000
 Name: salary_in_usd, dtype: float64)

In [9]:
for column in categorical_columns:
    data[column] = data[column].str.strip().str.lower()

for column in categorical_columns:
    data[column] = data[column].astype('category')

final_data_types = data.dtypes

final_data_types

work_year                int64
experience_level      category
employment_type       category
job_title             category
salary                   int64
salary_currency       category
salary_in_usd            int64
employee_residence    category
remote_ratio             int64
company_location      category
company_size          category
dtype: object

In [15]:
#1. What is the average salary in USD across all job titles, and how does it vary by experience level?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT experience_level, AVG(salary_in_usd) AS average_salary
FROM salaries
GROUP BY experience_level
ORDER BY average_salary DESC
'''

cursor.execute(query)

results = cursor.fetchall()

for row in results:
    print(f"Experience Level: {row[0]}, Average Salary (USD): {row[1]:.2f}")

Experience Level: EX, Average Salary (USD): 194930.93
Experience Level: SE, Average Salary (USD): 153051.07
Experience Level: MI, Average Salary (USD): 104525.94
Experience Level: EN, Average Salary (USD): 78546.28


In [16]:
#2. How does remote work affect salaries in the data science field?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT
    CASE
        WHEN remote_ratio = 0 THEN 'No Remote (On-site)'
        WHEN remote_ratio > 0 AND remote_ratio < 100 THEN 'Partially Remote'
        WHEN remote_ratio = 100 THEN 'Fully Remote'
    END AS remote_category,
    AVG(salary_in_usd) AS average_salary
FROM salaries
GROUP BY remote_category
ORDER BY average_salary DESC
'''

cursor.execute(query)

results = cursor.fetchall()

for row in results:
    print(f"Remote Category: {row[0]}, Average Salary (USD): {row[1]:.2f}")

Remote Category: No Remote (On-site), Average Salary (USD): 144316.20
Remote Category: Fully Remote, Average Salary (USD): 136481.45
Remote Category: Partially Remote, Average Salary (USD): 78400.69


In [17]:
#3. What are the top 5 job titles with the highest average salaries?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT job_title, AVG(salary_in_usd) AS average_salary
FROM salaries
GROUP BY job_title
ORDER BY average_salary DESC
LIMIT 5
'''

cursor.execute(query)

results = cursor.fetchall()

for row in results:
    print(f"Job Title: {row[0]}, Average Salary (USD): {row[1]:.2f}")

Job Title: Data Science Tech Lead, Average Salary (USD): 375000.00
Job Title: Cloud Data Architect, Average Salary (USD): 250000.00
Job Title: Data Lead, Average Salary (USD): 212500.00
Job Title: Data Analytics Lead, Average Salary (USD): 211254.50
Job Title: Principal Data Scientist, Average Salary (USD): 198171.12


In [18]:
#4. How do salaries vary by company size?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT company_size, AVG(salary_in_usd) AS average_salary
FROM salaries
GROUP BY company_size
ORDER BY average_salary DESC
'''

cursor.execute(query)

results = cursor.fetchall()

for row in results:
    print(f"Company Size: {row[0]}, Average Salary (USD): {row[1]:.2f}")

Company Size: M, Average Salary (USD): 143130.55
Company Size: L, Average Salary (USD): 118300.98
Company Size: S, Average Salary (USD): 78226.68


In [21]:
#5. What are the most common employment types (Full-time, Contract, etc.) in the dataset, and how do salaries vary among them?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT employment_type, AVG(salary_in_usd) AS average_salary, COUNT(*) AS num_positions
FROM salaries
GROUP BY employment_type
ORDER BY num_positions DESC, average_salary DESC
'''

cursor.execute(query)

results = cursor.fetchall()

print("Employment Type, Average Salary (USD), Number of Positions:")
for row in results:
    print(f"{row[0]}, {row[1]:.2f}, {row[2]}")

Employment Type, Average Salary (USD), Number of Positions:
FT, 138314.20, 3718
PT, 39533.71, 17
CT, 113446.90, 10
FL, 51807.80, 10


In [23]:
#6. How have salaries in the data science field changed over the years covered in the dataset?

conn = sqlite3.connect('ds_salaries.db')

cursor = conn.cursor()

query = '''
SELECT work_year, AVG(salary_in_usd) AS average_salary
FROM salaries
GROUP BY work_year
ORDER BY work_year ASC
'''

cursor.execute(query)

results = cursor.fetchall()

print("Average salary in USD by year:")
for row in results:
    print(f"Year: {row[0]}, Average Salary (USD): {row[1]:.2f}")

conn.close()

Average salary in USD by year:
Year: 2020, Average Salary (USD): 92302.63
Year: 2021, Average Salary (USD): 94087.21
Year: 2022, Average Salary (USD): 133338.62
Year: 2023, Average Salary (USD): 149045.54
